In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16) 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset_dir = '../input/challenge2/SharingFolder/Training.csv'
column_names = ['Sponginess', 'Wonder level', 'Crunchiness', 'Loudness on impact', 'Meme creativity', 'Soap slipperiness','Hype root']

df = pd.read_csv(dataset_dir, header=0, names=column_names)

In [ ]:
#senza test
X_train_raw = df
X_min = X_train_raw.min()
X_max = X_train_raw.max()
X_train_raw = (X_train_raw-X_min)/(X_max-X_min)



In [ ]:
np.array(X_min,dtype=np.float32)

In [ ]:
np.array(X_max,dtype=np.float32)

In [ ]:
window = 200
stride = 10
target_labels = df.columns
telescope = 1

In [ ]:
future = df[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [ ]:
def build_sequences(df, target_labels=['sponginess'], window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float32')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape #X_test.shape, y_test.shape

# Multivariate Forecasting (Autoregression)

In [ ]:
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [ ]:
inspect_multivariate(X_train, y_train, target_labels, telescope)

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 200

In [ ]:
input_shape

In [ ]:
output_shape

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=(input_shape), name='Input')
    
    
    output_train = tfkl.Input(shape=(output_shape), name='Input')
    
    # blocco 1 
    encoder_stack_h, encoder_last_h, encoder_last_c = tfkl.LSTM(64,return_state=True, return_sequences=True, recurrent_dropout=0.2,dropout = 0.2)(input_layer)
    
    #normalization
    encoder_last_h = tfkl.BatchNormalization(momentum=0.6)(encoder_last_h)
    encoder_last_c = tfkl.BatchNormalization(momentum=0.6)(encoder_last_c)
    
    decoder_input = tfkl.RepeatVector(output_train.shape[1])(encoder_last_h)
    
    
    decoder_stack_h = tfkl.LSTM(64,dropout=0.2, recurrent_dropout=0.2,
                             return_state=False, return_sequences=True)(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
    
    # dot product + softmax 
    attention = tfkl.dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
    attention = tfkl.Activation('softmax')(attention)
    
    #context vector 
    context = tfkl.dot([attention, encoder_stack_h], axes=[2,1])
    context = tfkl.BatchNormalization(momentum=0.6)(context)
    
    decoder_combined_context = tfkl.concatenate([context, decoder_stack_h])
    
    output_layer = tfkl.TimeDistributed(tfkl.Dense(output_shape[1]))(decoder_combined_context)
    
    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')
    
    # Compile the model
    model.compile(loss = tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae', 'mse', tfk.metrics.RootMeanSquaredError()])

    # Return the model
    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=0.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
model.save('trained/model10_LSTM+attention')
!zip -r DirectForecasting.zip  ./trained/DirectForecasting